In [1]:
%load_ext autoreload
%autoreload 2

import logging
from models import *
from client import IniesClient
from process import *

logging.basicConfig(
    level=logging.INFO, format="%(asctime)s [%(levelname)s] %(message)s"
)
logging.getLogger("httpx").setLevel(logging.WARNING)


In [2]:
client = IniesClient()
try:
    all_epds = await client.get_all_epds(datetime(2024, 12, 18))
finally:
    await client.close()

2024-12-20 12:40:20,320 [INFO] Logging in to https://base-inies.fr/ws/Login.


2024-12-20 12:40:20,433 [INFO] Fetching normes
2024-12-20 12:40:20,536 [INFO] Fetching indicators and phases
2024-12-20 12:40:21,405 [INFO] Fetching all EPDs (short)
2024-12-20 12:40:21,570 [INFO] Number of EPDs to retrieve: 71
Fetching EPDs: 100%|██████████| 71/71 [00:46<00:00,  1.53epd/s]


In [ ]:
client = IniesClient()
try:
    epd = await client.get_epd(41610)
finally:
    await client.close()

In [6]:
df = process_all_epds(all_epds)
df = modify_columns_names(df)
df.head()

,Name_FDES,ID_FDES,Unique_ID_FDES_version,Version,Issue_Date,Declaration_Type,Declaration_Type_Name,Norme,Organisme_Name,DVT,...,C1,C2,C3,C4,CarbonBiogenicStorage,Packaging_Carbone_Biogenic_Storage,Distance_Transport_A4_Km,Production_Place,Production_Region_FR,Constituant_Products
0,Daikin Altherma 3 R F 180l (chauffage et eau c...,DAIK-00001-V01.02-FR,41619,1.4,2022/07/01,1,Individual,PEP PCR ed 3.0,DAIKIN EUROPE N.V.,17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,None,Plastic 12.0 % ; Metal 76.1 % ; Other 11.9 %
1,Daikin Altherma 3 R F 230l (chauffage et eau c...,DAIK-00002-V01.02-FR,41618,1.4,2022/07/01,1,Individual,PEP PCR ed 3.0,DAIKIN EUROPE N.V.,17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,None,Plastic 11.9 % ; Metal 76.7 % ; Other 11.4 %
2,FOAMGLAS T3+,INIES_IFOA20180531_145626,41607,1.5,2021/01/18,1,Individual,NF EN 15804+A1 + NF EN 15804/CN,PITTSBURGH CORNING FRANCE,100,...,NaN,NaN,NaN,NaN,0.0,NaN,274.38,Europe,None,Palette en bois 0.156742093 kg ; Emballage 0.0...
3,"ClimaGuard®, SunGuard®, verres techniques Guar...",INIES_ICZS20211011_094748,41625,1.4,2022/06/15,1,Individual,NF EN 15804+A1 + NF EN 15804/CN,Guardian Europe Sàrl,30,...,NaN,NaN,NaN,NaN,0.0,NaN,1000.00,Europe,None,"Verres plats revêtus ClimaGuard®, SunGuard®, v..."
4,Remblais légers de granulats d’argile expansée...,INIES_IREM20210831_170919,41628,1.4,2024/10/31,1,Individual,NF EN 15804+A1 + NF EN 15804/CN,ARGEX,100,...,NaN,NaN,NaN,NaN,0.0,NaN,339.95,Europe,None,PE film 0.07378 kg ; Remblais légers de granul...


In [7]:
df.to_excel("./export_inies_small.xlsx")